# Demonstration of Synthius

This notebook displays the core functionality of Synthius, wrapped in a high-level programming interface

Essentially, Synthius consists of two steps at this stage:
1. Generation of Synthetic data given generation models and original data.
2. Evaluation of the synthetic data with various metrics as output.

The highest-level API allows both steps to be done with one call though it is designed to be loosely coupled and each step can be executed individually.



### Step 1 - Install Synthius

Run the following command in the CLI: `pip install synthius`.

### Step 2 - Prepare Your Data

Ensure your original data is in its own directory and in a csv file.

e.g.

Data directory: `/data`

Data filename: `data.csv`

Data absolute path: `/data/data.csv`

### Step 3 - Prepare Your Models

Note that Synthius exposes an interface which allows you to utilize any model that follows the Python protocol in `synthius/model/synthesizer.py`:

```
@runtime_checkable
class Synthesizer(Protocol):
    """Any synthetic data model should implement this interface."""
    metadata: Optional[SingleTableMetadata]
    name: str  # unique name for saving/output

    def fit(self, train_data: pd.DataFrame) -> None:
        ...
    
    def generate(self, total_samples: int, conditions: list = None) -> pd.DataFrame:
        ...
```

For our purposes, we wrap several methods to implement the above interface.

In [ ]:
from synthius.model import (
    SDVSynthesizer,
    SynthesizerGaussianMultivariate,
    ARFSynthesizer,
    WGANSynthesizer
)

from sdv.single_table import (
    CopulaGANSynthesizer,
    CTGANSynthesizer,
    GaussianCopulaSynthesizer,
    TVAESynthesizer,
)

models = [
    SDVSynthesizer(CopulaGANSynthesizer),
    SDVSynthesizer(CTGANSynthesizer),
    SDVSynthesizer(GaussianCopulaSynthesizer),
    SDVSynthesizer(TVAESynthesizer),
    SynthesizerGaussianMultivariate(results_path="/storage/Synthius/examples/synthetic_data/"),
    ARFSynthesizer(id_column=None),
    # WGANSynthesizer()  # optional
]

### Step 4 - Prepare Field Metadata

There are certain metadata that should be specified for the evaluation phase of Synthius. Notably, one must provide `key_fields`, `sensitive_fields`, and `aux_cols`.

These are:
 Description here

 

In [ ]:
# Fix to match adult!

key_fields = [
    "sepal length (cm)",
    "sepal width (cm)",
    "petal length (cm)",
    "petal width (cm)",
    "target",
    "target_binary",
]

sensitive_fields = ["petal width (cm)"]

aux_cols = [
    ["Occupation", "Education", "Education-num", "Hours-per-week", "Capital-loss", "Capital-gain"],
    ["Race", "Sex", "Fnlwgt", "Age", "Native-country", "Workclass", "Marital-status", "Relationship"],
]

### Step 5 - Run Synthius

Now we simply call the API and pass in the appropriate caching directories. 

Note the random seed only controls the train test split of the original data.

In [ ]:
from api import high_level

run_synthius(
    original_data_filename="data.csv",
    data_dir="/storage/Synthius/examples/data",
    synth_dir="/storage/Synthius/examples/synthetic_data",
    models_dir="/storage/Synthius/examples/models",
    results_dir="/storage/Synthius/examples/metrics",
    target_column="Income",
    key_fields=key_fields,
    sensitive_fields=sensitive_fields,
    aux_cols=aux_cols,
    models=models,
    random_seed=42
)